# Setup

In [16]:
import numpy as np
import seaborn as sns
import pandas as pd
import glob

# Functions

In [17]:
def read_csv(filepath, cols, names):
    df = pd.read_csv(filepath, usecols = cols, names = names, header = None)
    df = df.drop([0], axis = 0) # Remove first row (headers)
    df = df.astype('float64') # Cast data to floats
    
    return df

In [210]:
def get_class_df(file_path, df):
    class_name = '_'.join(file_path.split('\\')[-1].split('_')[:-1])

    return pd.DataFrame({'class': [class_name for _ in range(len(df))]})

In [211]:
def read_acc_mag_csvs():
    X = pd.DataFrame()
    y = pd.DataFrame(columns=['class'])
    
    for file in glob.glob('data/cleaned/acc_mag/*.csv'):

        if 'acc' in file:
            mag_file = file.replace(file.split('\\')[-1], file.split('\\')[-1].replace('acc', 'mag'))
            
            df_acc = read_csv(file, [1,2,3], ['ax', 'ay', 'az'])
            df_mag = read_csv(mag_file, [1,2,3], ['mx', 'my', 'mz'])
            
            df = pd.concat([df_acc, df_mag], axis=1, join='inner')
            X = X.append(df, ignore_index=True)            

            y = y.append(get_class_df(file, df), ignore_index=True)

    return X, y

In [214]:
def read_acc_gyro_csvs():
    X = pd.DataFrame()
    y = pd.DataFrame(columns=['class'])
    
    for file in glob.glob('data/cleaned/acc_gyro/*.csv'):

        if 'acc' in file:
            gyro_file = file.replace(file.split('\\')[-1], file.split('\\')[-1].replace('acc', 'gyro'))
            
            df_acc = read_csv(file, [1,2,3], ['ax', 'ay', 'az'])
            df_gyro = read_csv(gyro_file, [1,2,3], ['gx', 'gy', 'gz'])
            
            df = pd.concat([df_acc, df_gyro], axis=1, join='inner')
            X = X.append(df, ignore_index = True)
            
            y = y.append(get_class_df(file, df), ignore_index=True)
        
    return X, y

# Prepare data

## Trim data

## Read data sets

In [216]:
X_acc_mag, y_acc_mag = read_acc_mag_csvs()

In [53]:
X_acc_gyro, y_acc_gyro = read_acc_gyro_csvs()